In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from __future__ import print_function
from collections import OrderedDict

import os
import sys
import timeit
import pickle

import scipy.io as sio
import numpy as np
import tensorflow as tf

sys.path.insert(0, "/home/jay/DSN/tensorflow/ADSA/Model/")
import cnnet as nn
import util
import read_data as read_data
import ADSA_DNN_maxmin as ADSA
import DataPackage as dp
import AmazonReviewsFeaturePlot as fp

In [2]:
option='Amazon'
source_data, target_data, max_feature = read_data.read(option)
train_ftd_source, train_labeld_source = source_data[0]
valid_ftd_source, valid_labeld_source = source_data[1]
test_ftd_source, test_labeld_source = source_data[2]
    
train_ftd_target, train_labeld_target = target_data[0]
valid_ftd_target, valid_labeld_target = target_data[1]
test_ftd_target, test_labeld_target = target_data[2]

batch_size=[80,80]
x_dim=max_feature
y_dim=2
struct=ADSA.ADSA_struct()
struct.FE_struct.layer_struct=[max_feature,500,100]
struct.FE_struct.activation=[tf.nn.relu, tf.nn.relu]
#struct.FE_struct.activation=[tf.nn.sigmoid, tf.nn.sigmoid]
struct.FE_struct.batch_norm=[True, True]
struct.PSFE_struct.layer_struct=[max_feature,500,100]
struct.PSFE_struct.activation=[tf.nn.relu, tf.nn.relu]
#struct.PSFE_struct.activation=[tf.nn.sigmoid, tf.nn.sigmoid]
struct.PSFE_struct.batch_norm=[True,True]
struct.PTFE_struct.layer_struct=[max_feature,500,100]
struct.PTFE_struct.activation=[tf.nn.relu, tf.nn.relu]
#struct.PTFE_struct.activation=[tf.nn.sigmoid, tf.nn.sigmoid]
struct.PTFE_struct.batch_norm=[True,True]
struct.TC_struct.layer_struct=[100, 2]
struct.TC_struct.activation=['linear']
struct.TC_struct.batch_norm=[False]
struct.DC_struct.layer_struct=[100,10, 2]
struct.DC_struct.activation=[ tf.nn.relu, 'linear']
struct.DC_struct.batch_norm=[False,False]
struct.Sep_struct.layer_struct=[100, 3]
struct.Sep_struct.activation=['linear']
struct.Sep_struct.batch_norm=[False]

description='Amazon_DSN'

In [3]:
# Build the model graph
graph=tf.get_default_graph()
with graph.as_default():
    model = ADSA.ADSAModel(struct,x_dim, y_dim)
    learning_rate= tf.placeholder(tf.float32, [])
    
    pred_loss = tf.reduce_mean(model.pred_loss)
    domain_loss = tf.reduce_mean(model.domain_adapt_loss)
    sep_loss=tf.reduce_mean(model.domain_sep_loss)
    
    
    
    
    
    domain_adapt_classifier_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'domain_predict')
    adapt_opt=tf.train.MomentumOptimizer(learning_rate, 0.09).minimize(domain_loss, var_list=domain_adapt_classifier_vars)
    #adapt_opt=tf.train.AdamOptimizer(learning_rate=0.00005).minimize(domain_loss, var_list=domain_adapt_classifier_vars)
                            
    target_FE_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'shared_feature_extractor')
    gen_opt=tf.train.MomentumOptimizer(learning_rate,0.09).minimize(-domain_loss, var_list=target_FE_vars)
    #gen_opt=tf.train.AdamOptimizer(learning_rate=0.00005).minimize(-domain_loss, var_list=target_FE_vars)
                            
    domain_sep_classifier_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'domain_sep_classifier')
    sep_opt=tf.train.MomentumOptimizer(learning_rate,0.09).minimize(-sep_loss,var_list=domain_sep_classifier_vars)
    #sep_opt=tf.train.AdamOptimizer(learning_rate=0.00005).minimize(-sep_loss,var_list=domain_sep_classifier_vars)
                            
    source_private_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'source_private_feature_extractor')
    target_private_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'target_private_feature_extractor')
    sep_encoder_vars=target_FE_vars+source_private_vars+target_private_vars
    sep_gen_opt=tf.train.MomentumOptimizer(learning_rate,0.9).minimize(sep_loss,var_list=sep_encoder_vars)
    #sep_gen_opt=tf.train.AdamOptimizer(learning_rate=0.00005).minimize(sep_loss,var_list=sep_encoder_vars)
    
    task_d_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'label_predictor')
    pre_train_op=tf.train.AdamOptimizer(learning_rate=0.0005).minimize(pred_loss, var_list=target_FE_vars+task_d_vars)
    
    total_loss = pred_loss - domain_loss+0.05*sep_loss+tf.reduce_mean(tf.nn.l2_normalize(task_d_vars[0],1))
    dann_train_op = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(total_loss)
    #dann_train_op = tf.train.AdamOptimizer(learning_rate=0.00005).minimize(total_loss)
    
    # Evaluation
    source_correct_label_pred = tf.equal(tf.argmax(model.source_label, 1), tf.argmax(model.source_pred, 1))
    source_acc = tf.reduce_mean(tf.cast(source_correct_label_pred, tf.float32))
    
    target_correct_label_pred = tf.equal(tf.argmax(model.target_label, 1), tf.argmax(model.target_pred, 1))
    target_acc = tf.reduce_mean(tf.cast(target_correct_label_pred, tf.float32))
    
    correct_domain_pred = tf.equal(tf.argmax(model.domain_labels, 1), tf.argmax(model.domain_adapt_pred, 1))
    domain_acc = tf.reduce_mean(tf.cast(correct_domain_pred, tf.float32))
                            
    correct_sep_pred=tf.equal(tf.argmax(model.domain_sep_labels, 1), tf.argmax(model.domain_sep_pred, 1))
    sep_acc=tf.reduce_mean(tf.cast(correct_sep_pred, tf.float32))
    
    #get feauture
    latent_feats=model.get_feature()

Shared_Feature_extractor is constructed with hidden layer number 1
Source_private_feature_extractor is constructed with hidden layer number 1
Target_private_feature_extractor is constructed with hidden layer number 1
Task_classifier is constructed with hidden layer number 0
Domain_adapt_classifier is constructed with hidden layer number 1
Domain_sep_classifier is constructed with hidden layer number 0


In [4]:
num_epoch=2000
def train_and_evaluate(training_mode, graph, model, verbose=False):
    
    with tf.Session(graph=graph) as sess:
        tf.global_variables_initializer().run()
        
        p_source_batch=util.batch_generator([train_ftd_source, train_labeld_source], batch_size[0])
        p_target_batch=util.batch_generator([train_ftd_target, train_labeld_target], batch_size[1])
        for i in range(1000):
            X0, y0= p_source_batch.next()
            X1, y1= p_target_batch.next()
            X=np.vstack([X0,X1])
            y=np.vstack([y0,y1])
            _, batch_loss, s_acc=\
                        sess.run([pre_train_op, pred_loss, source_acc], feed_dict={model.X: X,model.y:y,
                                                                        model.batch_size: batch_size,
                                                                        model.train_flag: False,
                                                                        model.flip_scale: 0,
                                                                        learning_rate:0.05})
            #print('    pre loss:%f,  source acc %f'%(batch_loss,np.mean(s_acc)))
        
        # Batch generators
        t_source_batch=util.batch_generator([train_ftd_source, train_labeld_source], batch_size[0])
        t_target_batch=util.batch_generator([train_ftd_target, train_labeld_target], batch_size[1])
        
        domain_labels=np.vstack([np.tile([0.9, 0.1], [batch_size[0], 1]), np.tile([0.1, 0.9],[batch_size[1], 1])])
        domain_sep_labels=np.vstack([np.tile([0.8, 0.1, 0.1], [batch_size[0]+batch_size[1], 1]), np.tile([0.1, 0.8, 0.1],[batch_size[0], 1]), np.tile([0.1, 0.1, 0.8],[batch_size[1], 1])])
        
        for itr in range(num_epoch):
            p = float(itr) / num_epoch
            flip_scale= 2. / (1. + np.exp(-10. * p)) 
            #lr=0.01 / (1. + 10 * p)**0.75
            lr=0.1/(1.+10*p)**0.75
            if training_mode=='dann':
                
                d_source_batch=util.batch_generator([train_ftd_source, train_labeld_source], batch_size[0])
                d_target_batch=util.batch_generator([train_ftd_target, train_labeld_target], batch_size[1])
                gen_source_batch= util.batch_generator([train_ftd_source, train_labeld_source], batch_size[0])
                gen_target_batch= util.batch_generator([train_ftd_target, train_labeld_target], batch_size[1])
                sep_source_batch= util.batch_generator([train_ftd_source, train_labeld_source], batch_size[0])
                sep_target_batch= util.batch_generator([train_ftd_target, train_labeld_target], batch_size[1])
                
                for d_iter in range(40):
                    dX0, dy0= d_source_batch.next()
                    dX1, dy1= d_target_batch.next()
                    dX=np.vstack([dX0,dX1])
                    dy=np.vstack([dy0, dy1])
                
                    _, d_loss=sess.run([adapt_opt, domain_loss], feed_dict={model.X: dX, model.y: dy,
                                                                            model.domain_labels: domain_labels,
                                                                            model.batch_size: batch_size,
                                                                            model.train_flag: True,
                                                                            model.flip_scale: flip_scale,
                                                                            learning_rate: lr})
                   
                for d_iter in range(2):
                    sX0, sy0 =sep_source_batch.next()
                    sX1, sy1 = sep_target_batch.next()
                    sX=np.vstack([sX0, sX1])
                    sy=np.vstack([sy0, sy1])
                    _, s_loss=sess.run([sep_opt, sep_loss], feed_dict={model.X: sX, model.y: sy,
                                                                       model.domain_labels: domain_labels,
                                                                       model.domain_sep_labels: domain_sep_labels,
                                                                       model.batch_size: batch_size,
                                                                       model.train_flag: True,
                                                                       model.flip_scale: flip_scale,
                                                                       learning_rate: 0.005})
                
                #print('   train D domain loss : %f\n'% d_loss)
                for g_iter in range(4):
                    gX0, gy0 =gen_source_batch.next()
                    gX1, gy1 = gen_target_batch.next()
                    gX=np.vstack([gX0, gX1])
                    gy=np.vstack([gy0, gy1])
                    _, g_loss=sess.run([gen_opt, domain_loss],feed_dict={model.X: gX, model.y: gy,
                                                                         model.domain_labels: domain_labels,
                                                                         model.batch_size: batch_size,
                                                                         model.train_flag: True,
                                                                         model.flip_scale: flip_scale,
                                                                         learning_rate: lr})
                
                for g_iter in range(2):
                    sX0, sy0 =sep_source_batch.next()
                    sX1, sy1 = sep_target_batch.next()
                    sX=np.vstack([sX0, sX1])
                    sy=np.vstack([sy0, sy1])
                    _, s_loss=sess.run([sep_gen_opt, sep_loss],feed_dict={model.X: sX, model.y: sy,
                                                                          model.domain_sep_labels: domain_sep_labels,
                                                                          model.batch_size: batch_size,
                                                                          model.train_flag: True,
                                                                          model.flip_scale: flip_scale,
                                                                          learning_rate: lr})
                   
                #print('   train G domain loss : %f\n'% g_loss)
                X0, y0 =t_source_batch.next()
                X1, y1 = t_target_batch.next()
                X=np.vstack([X0, X1])
                y=np.vstack([y0, y1])
                _, batch_loss, p_acc, d_acc ,s_acc=\
                        sess.run([dann_train_op, total_loss,source_acc, domain_acc,sep_acc], feed_dict={model.X: X,model.y:y,
                                                                                               model.domain_labels:domain_labels,
                                                                                               model.domain_sep_labels: domain_sep_labels,
                                                                                               model.batch_size: batch_size,
                                                                                               model.train_flag: True,
                                                                                               model.flip_scale: flip_scale,
                                                                                               learning_rate:lr})
                
                if verbose and itr % 100==0:
                    print ('epoch %i  loss: %f adapt_acc: %f task_acc: %f  sep acc: %f \n'
                           '          flip_scale: %f  learning_rate: %f' %
                           (itr,batch_loss, d_acc, p_acc, s_acc, flip_scale, lr))
                    
                if verbose and itr%1000==0:
                    val_size=[valid_ftd_source.shape[0],valid_ftd_target.shape[0]]
                    val_data=np.vstack([valid_ftd_source, valid_ftd_target])
                    val_label=np.vstack([valid_labeld_source, valid_labeld_target])
                    val_s_acc, val_t_acc=\
                        sess.run([source_acc, target_acc], feed_dict={model.X: val_data,model.y:val_label,
                                                                     model.batch_size: val_size,
                                                                     model.train_flag: False,
                                                                    })
                    print('      val source acc: %f  target acc: %f \n'% (val_s_acc, val_t_acc))
                
        # test
        test_size=[test_ftd_source.shape[0],test_ftd_target.shape[0]]
        test_data=np.vstack([test_ftd_source,test_ftd_target])
        test_label=np.vstack([test_labeld_source, test_labeld_target])
        
        
        
        
        source_ac, target_ac,l_f= sess.run([source_acc, target_acc,latent_feats],
                            feed_dict={model.X: test_data, model.y: test_label,
                                       model.batch_size: test_size, model.train_flag: False})
        
       
        
    return source_ac*100, target_ac*100, l_f

In [5]:
print ('\nDomain adaptation training')
source_acc, target_acc, feats= train_and_evaluate('dann', graph, model,True)
print ('Source  accuracy: %f'% source_acc)
print ('Target  accuracy: %f'% target_acc)


Domain adaptation training
epoch 0  loss: -0.541673 adapt_acc: 0.843750 task_acc: 1.000000  sep acc: 0.409375 
          flip_scale: 1.000000  learning_rate: 0.100000
      val source acc: 0.870000  target acc: 0.710000 

epoch 100  loss: -1.309022 adapt_acc: 0.631250 task_acc: 1.000000  sep acc: 1.000000 
          flip_scale: 1.244919  learning_rate: 0.073779
epoch 200  loss: -1.360310 adapt_acc: 0.518750 task_acc: 1.000000  sep acc: 1.000000 
          flip_scale: 1.462117  learning_rate: 0.059460
epoch 300  loss: -1.317948 adapt_acc: 0.568750 task_acc: 1.000000  sep acc: 1.000000 
          flip_scale: 1.635149  learning_rate: 0.050297
epoch 400  loss: -1.315751 adapt_acc: 0.631250 task_acc: 1.000000  sep acc: 1.000000 
          flip_scale: 1.761594  learning_rate: 0.043869
epoch 500  loss: -1.327483 adapt_acc: 0.575000 task_acc: 1.000000  sep acc: 1.000000 
          flip_scale: 1.848284  learning_rate: 0.039079
epoch 600  loss: -1.345623 adapt_acc: 0.568750 task_acc: 1.000000  

In [6]:
#sample_n=2000
#fp.features_plot(feats,source_data, target_data,sample_n, description)